This notebook checks the behaviour of the embedding space

In [1]:
import re
import os
import sys
import pickle
import random
import glob
import scipy.sparse

import numpy as np
import pandas as pd

from annoy import AnnoyIndex
from scipy.spatial import distance

scriptpath = "../../"
sys.path.append(os.path.abspath(scriptpath))
from helpers.helpers_channels_more_300 import *

In [11]:
'''
Retrieve the array obtained by apllying the dimentionality reduction algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETER:
    - file_path: the path where the embedding graph is stored

RETURN: 
    - df: DataFrame representing the graph in the embedding space
'''
def get_dataframe_in_embedding_space(file_path):
    graph_matrix = np.load(file_path)
    df = pd.DataFrame(graph_matrix)
    df = df.rename(lambda x: 'dr'+str(x), axis='columns')
    df['index'] = np.load('/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/channels_more_300/size_200_sub_0043_neg_35_threshold_20/channel_embeddingindex_ordering.npy').astype(int)
    df = df.sort_values(by=['index'])
    return df.set_index('index')


In [5]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

### Find k closest channels using annoy library

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we should get similar channels if the embedding space is good.

In [13]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/channelcrawler.csv")
channelcrawler['channel_id'] = channelcrawler['link'].str.split('/').str[-1]
channelcrawler.head()

,category,join_date,link,name,subscribers,videos,channel_id
0,Film and Animation,2017-05-21,http://www.youtube.com/channel/UCBJuEqXfXTdcPS...,MagnusNation,65100,28,UCBJuEqXfXTdcPSbGO9qqn1g
1,Entertainment,2011-12-13,http://www.youtube.com/channel/UCkNW9Q1VR_aeZ6...,Mago Dario Animazion...,60200,48,UCkNW9Q1VR_aeZ6uht83jJVQ
2,Music,2013-09-13,http://www.youtube.com/channel/UC1xcnrpcF59FWW...,Mägo de Oz - Topic,40200,395,UC1xcnrpcF59FWWELtZvJTdg
3,Music,2008-03-17,http://www.youtube.com/channel/UCXhkGgooXHDNwg...,Mago Merlino,14800,838,UCXhkGgooXHDNwgJXmoTSN7g
4,Entertainment,2014-10-19,http://www.youtube.com/channel/UCvZGsuvKlYOGiZ...,MAGO TOMÁS,26200,31,UCvZGsuvKlYOGiZTsxwJNS5Q


In [14]:
channelcrawler = channelcrawler[channelcrawler['channel_id'].apply(lambda row: row in channels_id)]

In [15]:
channelcrawler['category'].unique()

array(['Film and Animation', 'Music', 'Comedy', 'Gaming',
       'Science & Technology', 'Sports', 'Entertainment', 'Education',
       'Nonprofits & Activism', 'People & Blogs', 'Howto & Style',
       'News & Politics', 'Travel & Events', 'Autos & Vehicles',
       'Pets & Animals', nan], dtype=object)

In [6]:
'''
    Retrieve the array obtained by apllying the dimentinality reductin algorithm
    graph_matrix: SHAPE: (channels, n_comp)
    
    PARAMETERS:
        - df_embedding: DataFrame representing the graph in the embedding space
        
    RETURN: The annoy index
    '''
def get_annoy_index(df):
    index = AnnoyIndex(df.shape[1], "euclidean")  # Length of item vector that will be indexed
    df.apply(lambda row: index.add_item(row.name, np.array(row)), axis = 1)
    index.build(100) # 100 trees
    return index

In [7]:
def get_k_nearest_neighbors(path, ref_index_channel, k = 20):
    df = get_dataframe_in_embedding_space(path)
    index = get_annoy_index(df)
    nearest_neighbors_index = index.get_nns_by_item(ref_index_channel, k, search_k = 100000000)
    nearest_neighbors_id = [dict_ind_channel[val] for val in nearest_neighbors_index]
    return nearest_neighbors_id

In [10]:
dict_channel_ind['UCLFW3EKD2My9swWH4eTLaYw']

19769

In [19]:
path = '/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/channels_more_300/size_200_sub_0043_neg_35_threshold_20/channel_embedding.npy'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCZyCposXwcyopaACep44maQ'], k= 30)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)

In [20]:
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCZyCposXwcyopaACep44maQ,Howto & Style,2011-09-29,http://www.youtube.com/channel/UCZyCposXwcyopa...,Alex Costa,2120000,543
1,UCbvIIQc5Jo9-jIXnkPe03oA,People & Blogs,2015-09-19,http://www.youtube.com/channel/UCbvIIQc5Jo9-jI...,Matti Haapoja,749000,346
2,UCvcm_wQUH4jvM33JgwVJ5Ew,Autos & Vehicles,2014-09-02,http://www.youtube.com/channel/UCvcm_wQUH4jvM3...,Roads Untraveled,224000,408
3,UCjo7oMbNu181jPBs3_lZIJA,Music,2014-06-13,http://www.youtube.com/channel/UCjo7oMbNu181jP...,Buffet Boys,680000,156
4,UCbjw-VrJCUb8PTGoiDI2UMw,Autos & Vehicles,2016-01-03,http://www.youtube.com/channel/UCbjw-VrJCUb8PT...,iamjake,153000,510
5,UCUpC8yomtIEfrTXLjk-wAmw,Autos & Vehicles,2006-08-07,http://www.youtube.com/channel/UCUpC8yomtIEfrT...,DoctaM3,128000,1145
6,UCCxawQ5mHZEPP9SvMFar1RA,Autos & Vehicles,2015-05-06,http://www.youtube.com/channel/UCCxawQ5mHZEPP9...,More Skids,146000,365
7,UCaxW6r282iWvzJmr3BwGc-A,Autos & Vehicles,2015-10-08,http://www.youtube.com/channel/UCaxW6r282iWvzJ...,Auto Addiction,305592,331
8,UCvhnYODy6WQ0mw_zi3V1h0g,Science & Technology,2011-10-11,http://www.youtube.com/channel/UCvhnYODy6WQ0mw...,JuanBagnell,94189,1139
9,UCQIyqMWCdx1GBvbw_Yi6lEA,Sports,2006-03-16,http://www.youtube.com/channel/UCQIyqMWCdx1GBv...,Aston Martin Red Bul...,547456,604


In [22]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCAql2DyGU2un1Ei2nMYsqOA'], k= 30)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)

In [23]:
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAql2DyGU2un1Ei2nMYsqOA,News & Politics,2015-03-17,http://www.youtube.com/channel/UCAql2DyGU2un1E...,Donald J Trump,189000,222
1,UCIVk1L1-JmpdiGuZcVjImtA,News & Politics,2008-10-09,http://www.youtube.com/channel/UCIVk1L1-JmpdiG...,ABC15 Arizona,175000,71190
2,UCi4fcBVyo4CAnmdgXeO-NvA,News & Politics,2014-02-27,http://www.youtube.com/channel/UCi4fcBVyo4CAnm...,CBS Boston,63500,31634
3,UC3-XP_zRye-BywerpbuOuCg,Travel & Events,2011-10-27,http://www.youtube.com/channel/UC3-XP_zRye-Byw...,Ed,35900,237
4,UCZm2IEBVHdtaAhSyYhWu9mg,Nonprofits & Activism,2007-09-10,http://www.youtube.com/channel/UCZm2IEBVHdtaAh...,InstituteForJustice,51100,307
5,UCE3AiPpvRrUQJ7qFiEkDhLw,Entertainment,2014-08-17,http://www.youtube.com/channel/UCE3AiPpvRrUQJ7...,ITS AGTV,43400,454
6,UCsK80SSNnnGkb2pwPgblMsA,Sports,2013-11-20,http://www.youtube.com/channel/UCsK80SSNnnGkb2...,James Lynch,10000,2784
7,UCruQg25yVBppUWjza8AlyZA,News & Politics,2007-10-18,http://www.youtube.com/channel/UCruQg25yVBppUW...,FOX 4 News - Dallas-...,170000,15263
8,UC34avTJVyhA6Af5-fZhmxgw,News & Politics,2011-01-24,http://www.youtube.com/channel/UC34avTJVyhA6Af...,Arana Nation,13600,605
9,UCI1H2wxkvshpCcUNNOEc4Gw,Travel & Events,2012-01-17,http://www.youtube.com/channel/UCI1H2wxkvshpCc...,IBTimes UK,115000,12331


In [24]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UC9pXxdNqCc2zjgRXSoowNNg'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UC9pXxdNqCc2zjgRXSoowNNg,News & Politics,2014-07-09,http://www.youtube.com/channel/UC9pXxdNqCc2zjg...,A Blue Dot In Texas,10600,646
1,UC4G1zEStJMqN3mnEONXA7Mw,People & Blogs,2014-05-26,http://www.youtube.com/channel/UC4G1zEStJMqN3m...,Bravo Von Muller,30200,1737
2,UCN_qIhm7BAq9Qxa6j9XMVXA,News & Politics,2017-02-16,http://www.youtube.com/channel/UCN_qIhm7BAq9Qx...,Breaking News TV,118000,439
3,UCglQOK6wX00WmP_Cpju2dGg,News & Politics,2010-10-12,http://www.youtube.com/channel/UCglQOK6wX00WmP...,SavageNationLiberty,20549,1350
4,UCudqVyQYnmi8NwWfd11Rmvw,Education,2009-02-06,http://www.youtube.com/channel/UCudqVyQYnmi8Nw...,Hillsdale College,77900,707
5,UCe3PDv7F4Uex3ii9zdMbvWQ,Travel & Events,2006-06-29,http://www.youtube.com/channel/UCe3PDv7F4Uex3i...,Reekay's LifeBeyondT...,45800,923
6,UCIn0uU_jcw8ge4jz1Dg9kng,Sports,2012-03-03,http://www.youtube.com/channel/UCIn0uU_jcw8ge4...,schlegdaddytv,12388,2112
7,UCwuA3ZQBPSNLLuHJIQSqtRA,Howto & Style,2012-04-20,http://www.youtube.com/channel/UCwuA3ZQBPSNLLu...,Aishwarya,96667,248
8,UC2GQkhnB8tQ_9P6p3lUYsfQ,Education,2011-07-20,http://www.youtube.com/channel/UC2GQkhnB8tQ_9P...,Dirtpatcheaven,128000,2003
9,UCCs_FjJR8A7rompHCnW0-3g,News & Politics,2007-10-24,http://www.youtube.com/channel/UCCs_FjJR8A7rom...,Gerald Celente,91500,634


In [25]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCAuUUnT6oDeKwE6v1NGQxug'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAuUUnT6oDeKwE6v1NGQxug,People & Blogs,2006-12-06,http://www.youtube.com/channel/UCAuUUnT6oDeKwE...,TED,14800000,3105
1,UCrTNhL_yO3tPTdQ5XgmmWjA,Entertainment,2007-09-29,http://www.youtube.com/channel/UCrTNhL_yO3tPTd...,RedLetterMedia,1050000,582
2,UCsT0YIqwnpJCM-mx7-gSA4Q,Nonprofits & Activism,2009-06-23,http://www.youtube.com/channel/UCsT0YIqwnpJCM-...,TEDx Talks,20700000,141627
3,UCLXo7UDZvByw2ixzpQCufnA,News & Politics,2014-03-04,http://www.youtube.com/channel/UCLXo7UDZvByw2i...,Vox,6570000,1048
4,UCswH8ovgUp5Bdg-0_JTYFNw,Education,2007-01-01,http://www.youtube.com/channel/UCswH8ovgUp5Bdg...,Russell Brand,1890000,1000
5,UCWJHDMgKWWvOsdyRF3HPVEw,Science & Technology,2008-02-26,http://www.youtube.com/channel/UCWJHDMgKWWvOsd...,Iraqveteran8888,2210000,1452
6,UCfGmaA-nXPryTfimsnkLieQ,Entertainment,2013-12-07,http://www.youtube.com/channel/UCfGmaA-nXPryTf...,Chibi Reviews,373000,7048
7,UCVtEytgcL5fZcSiKx-BjimQ,Gaming,2007-03-01,http://www.youtube.com/channel/UCVtEytgcL5fZcS...,AlphaOmegaSin,463806,1058
8,UCUsN5ZwHx2kILm84-jPDeXw,Comedy,2006-06-14,http://www.youtube.com/channel/UCUsN5ZwHx2kILm...,Comedy Central,8540000,4818
9,UCxXUQuvoiIAlpM2osoAitjQ,Entertainment,2009-10-14,http://www.youtube.com/channel/UCxXUQuvoiIAlpM...,Matt Jarbo,132000,5308


In [26]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCwTkM6CvIsYFaFiMKIKCqHw'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCwTkM6CvIsYFaFiMKIKCqHw,Film and Animation,2005-11-27,http://www.youtube.com/channel/UCwTkM6CvIsYFaF...,James Bond 007,115000,407
1,UC4E4CgNyGPAX9HJH8eEJAdw,Education,2007-11-26,http://www.youtube.com/channel/UC4E4CgNyGPAX9H...,University of Gronin...,22743,1847
2,UCoV_kw2gLbp2MtxFq5WU4wA,Gaming,2007-08-18,http://www.youtube.com/channel/UCoV_kw2gLbp2Mt...,Storm Dain Productio...,10424,202
3,UCVsnBJuq9hXGl8_BzZbuFuQ,Gaming,2009-07-27,http://www.youtube.com/channel/UCVsnBJuq9hXGl8...,scoutthedoggie2002,60300,166
4,UC1pOTJteEef10zJM0cHs4iQ,Autos & Vehicles,2008-03-05,http://www.youtube.com/channel/UC1pOTJteEef10z...,Toyota USA,370000,2768
5,UCFBeIDDRrmE3uP1_bCATjUA,Entertainment,2006-05-10,http://www.youtube.com/channel/UCFBeIDDRrmE3uP...,Killer Orange Cat,33900,289
6,UCZJjZTHRaG1NFYrrUEWZbLg,Music,2010-01-06,http://www.youtube.com/channel/UCZJjZTHRaG1NFY...,sarper duman,99500,72
7,UCerEIdrEW-IqwvlH8lTQUJQ,Science & Technology,2011-07-09,http://www.youtube.com/channel/UCerEIdrEW-Iqwv...,AkBKukU,60400,144
8,UCD4sZPWd3SZFeMidqBBWAFQ,Gaming,2012-11-16,http://www.youtube.com/channel/UCD4sZPWd3SZFeM...,TonyBingGaming,20780,1207
9,UCfpw11tqWqXUlJ_DrO-IHdA,Music,2012-07-24,http://www.youtube.com/channel/UCfpw11tqWqXUlJ...,HalusaTwin,14200,801


In [27]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,http://www.youtube.com/channel/UCISF5OGuAtSLNF...,Magpiepony,736000,304
1,UCtweSAEAMmIWwJxGEQdgpPA,Film and Animation,2016-07-19,http://www.youtube.com/channel/UCtweSAEAMmIWwJ...,Stariaat,977000,124
2,UC94Z4HZJkhPm94YPH1GE3bw,Entertainment,2011-12-13,http://www.youtube.com/channel/UC94Z4HZJkhPm94...,EileMonty,504000,211
3,UCawR3bBZNyQd4wPeuedGOGQ,Gaming,2014-07-22,http://www.youtube.com/channel/UCawR3bBZNyQd4w...,CookieGuy,382000,245
4,UC7EFWpvc1wYuUwrtZ_BLi9A,Education,2015-01-26,http://www.youtube.com/channel/UC7EFWpvc1wYuUw...,Kids Learning Tube,539000,278
5,UCpmC_0RGvkopa2r0nx8K8kQ,Gaming,2015-09-16,http://www.youtube.com/channel/UCpmC_0RGvkopa2...,Fat Paps,298000,39
6,UCuxTG36WkQ1FrxkfMIpSs_w,Gaming,2010-04-07,http://www.youtube.com/channel/UCuxTG36WkQ1Frx...,Silver WillowWing,115000,960
7,UCwV-qIQCH2i24TfLK8GGEYA,Pets & Animals,2010-11-29,http://www.youtube.com/channel/UCwV-qIQCH2i24T...,N2 Cat Crew,149000,200
8,UCEdAKZR4uJ5judsslc9TP_w,Gaming,2013-04-15,http://www.youtube.com/channel/UCEdAKZR4uJ5jud...,Thinks Wife,118000,53
9,UCWQS7LilJ64Uh_kShgF4gAg,Gaming,2015-02-02,http://www.youtube.com/channel/UCWQS7LilJ64Uh_...,twomanyraptors,178000,406


### Channels selected over the whole comments dataset
We randomly choose 10 000 users over the dataset.
For each user, we then pick two channels at random in the set of channels this user commented in.

In [14]:
selected_users_rw = np.random.randint(227495283-1, size=3000)

In [5]:
for v1, v2 in selected_channels_rw:
    if v1 == v2:
        print('haha')

In [15]:
selected_channels_rw = np.random.randint(129616-1, size=(3000, 2))

In [12]:
1 % 2

1

In [16]:
# Load the channel tuple sparse matrix
#S = scipy.sparse.load_npz('/dlabdata1/youtube_large/jouven/final_sparse_matrix/channels_more_300/sparse_matrix_for_word2vec.npz')
NB_SAMPLE = 3000
'''
with open("/dlabdata1/youtube_large/jouven/channels_more_300/selected_users_word2vecf.pkl",'rb') as f:
     selected_users = pickle.load(f)
f.close()
selected_users = np.random.choice(selected_users, NB_SAMPLE, replace=False)

S = S[:, selected_users]
# Create and store channels tuples
channels_tuple = []
for i in range(S.shape[1]):
    idx = S[:, i].nonzero()
    idx = idx[0]
    
    selected_channels = np.random.choice(idx, 2, replace=False)
    channels_tuple.append((selected_channels[0], selected_channels[1]))
    
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_user_walk_known_users.pkl",'wb') as f:
     pickle.dump(channels_tuple, f)
f.close()

S = scipy.sparse.load_npz('/dlabdata1/youtube_large/jouven/final_sparse_matrix/channels_more_300/sparse_matrix_word2vec_users_commented_geq_2_channels.npz')
'''
S2 = S[:, selected_users_rw]

# Create and store channels tuples
channels_tuple = []
last = 0
for i in range(S2.shape[1]):
    idx = S2[:, i].nonzero()
    idx = idx[0]
    
    if i % 2 == 1:
        selected_channel = np.random.choice(idx, 1)
        channels_tuple.append((last, selected_channel))
    else:
        last = np.random.choice(idx, 1)
    
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_random_walk_modified.pkl",'wb') as f:
     pickle.dump(channels_tuple, f)
f.close()

channels_tuple = []
for val in selected_channels_rw:
    channels_tuple.append((val[0], val[1]))
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_random_walk.pkl",'wb') as f:
     pickle.dump(channels_tuple, f)
f.close()


# Sample NB_SAMPLE from the set of users
selected_users = np.random.randint(S.shape[1] - 1, size=NB_SAMPLE)
S = S[:, selected_users]

# Create and store channels tuples
channels_tuple = []
for i in range(S.shape[1]):
    idx = S[:, i].nonzero()
    idx = idx[0]
    
    selected_channels = np.random.choice(idx, 2, replace=False)
    channels_tuple.append((selected_channels[0], selected_channels[1]))
    
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_user_walk.pkl",'wb') as f:
     pickle.dump(channels_tuple, f)
f.close()



In [6]:
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_user_walk_known_users.pkl",'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

#### Random walk

In [7]:
def get_random_walk(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_random_walk.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

In [8]:
def get_random_walk_modified(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_random_walk_modified.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

#### Compute metrics: users walk distance and relative nearest neighbor ranking 

In this section we want to measure the euclidian distance of a user walk compared to a random walk.

user walk: Euclidean distance in the embedding space between two randomly channels of a user.
random walk: Euclidean distance in the embedding space between two randomly channels.
position: Position of a channel taken from user u relatively of another channel taken from the same user in terms of its nearest neighbor ranking.


In [9]:
'''
Get the position of ref_channel relative to second_channel in terms of its nearest neighbors ranking.
PARAMETER:
    - ref_channel: The reference channel on which wwe compute it's k nearest neighbor
    - second_channel: The channel where we compute it's ranking relatively to ref_channel
    - dist: Euclidean distance between ref_channel and second_channel
    - index: annoy index
    - df_embedding: DataFrame representing the embedding space

RETURN: The position of second_channel relatively to ref_channel in terms of it's ranking

'''
def get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding):
    
    nearest_neighbors_index = index.get_nns_by_item(ref_channel, len(df_embedding), search_k = 100000000)
    dist_k_th_nearest = distance.euclidean(df_embedding.iloc[ref_channel], 
                                           df_embedding.iloc[nearest_neighbors_index[len(nearest_neighbors_index)-1]])
    for i in range(0, len(nearest_neighbors_index)):
        if nearest_neighbors_index[i] == second_channel:
            return i
    

In [10]:
users_walk_tab = []
ranking_position_tab = []

len_random_set = len(channels_tuple)
len_embedding = len(channels_id)
 
files = ['/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/channels_more_300/size_200_sub_0043_neg_35_threshold_10/channel_embedding.npy']
for file in files: 
    print('file ', file)
    df_embedding = get_dataframe_in_embedding_space(file)
    n_comp = df_embedding.shape[1]
    print('n_comp ', n_comp)
    random_walk_distance = get_random_walk(df_embedding)
        
    index = get_annoy_index(df_embedding)

    users_walk = 0
    ranking_position = 0

    for ref_channel, second_channel in channels_tuple:
        dist = distance.euclidean(df_embedding.iloc[ref_channel], df_embedding.iloc[second_channel])
        users_walk += dist
        ranking_position += get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding)
    
    users_walk_tab.append(users_walk/random_walk_distance)
    ranking_position_tab.append(ranking_position/(len_random_set*len_embedding))


file  /dlabdata1/youtube_large/jouven/word2vecf_preprocessing/channels_more_300/size_200_sub_0043_neg_35_threshold_10/channel_embedding.npy
n_comp  200


In [11]:
users_walk_tab

[2.593458525824739]

In [12]:
ranking_position_tab

[0.5553063510677694]

In [11]:
users_walk_tab

[2.2101317966239815]

In [12]:
ranking_position_tab

[0.555226085771304]

In [ ]:
'''
Plot the results obtained when computing the metrics
'''
import plotly.graph_objects as go

components_name = ['Features dim 50', 'Features dim 100', 'Features dim 200', 'Features dim 500']
colors = ['b', 'g,', 'r', 'c']

fig = go.Figure()

for i in range(len(components_name)):
    fig.add_trace(go.Scatter(
        x=ranking_position_tab,
        y=users_walk_tab,
        marker=dict(color=colors[i], size=12),
        mode="markers",
        name=components_name[i],
    ))

fig.update_layout(title="Gender Earnings Disparity",
                  xaxis_title="Ranking position",
                  yaxis_title="Users walk distance")

fig.show()